# Practical SQL

This Jupyter notebook will display all SQL queries that I have created as answers to the challenge questions at the end of each chapter of the book "Practical SQL: A Beginner's Guide To Storytelling With Data" by Anthony DeBarros.

The first part of this notebook will require us to import a couple of dependencies as well as personal data that will allow us to link out Jupyter Notebook to PostgreSQL.

In [1]:
import psycopg2
import pandas as pd
from sql_data import db, usr, pwd

In [2]:
# Connecting to postgreSQL database
conn = psycopg2.connect(
    host = "localhost",
    database = db,
    user = usr,
    password = pwd,
    port = 5432
)

def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        column_names = [i[0] for i in cursor.description]
        results = pd.DataFrame(results, columns= column_names)
        return results
        print("Query executed succesfully!")
        # Closing the cursor
        cursor.close()
        del cursor
        # Closing the connection
        connection.close()
    except OperationalError as e:
        print(f"The error '{e}' occurred.")      

## Chapter 2: Beginning Data Exploration With Select

Challenge Questions

1. Write a query that lists the schools in alphabetical order along with teachers ordered by last name A-Z

In [3]:
query = """
SELECT school, first_name, last_name
FROM teachers
ORDER BY school, last_name;
"""

execute_query(conn, query)

,school,first_name,last_name
0,F.D. Rossevelt HS,Lee,Reynolds
1,F.D. Rossevelt HS,Kathleen,Roush
2,F.D. Rossevelt HS,Janet,Smith
3,Myers Middle School,Samantha,Bush
4,Myers Middle School,Samuel,Cole
5,Myers Middle School,Betty,Diaz


2. Write a query that finds the one teacher whose first name starts with the letter S and who earns more that $40k


In [4]:
query = """
SELECT *
FROM teachers
WHERE salary > 40000 AND
	first_name ILIKE 's%';
"""

execute_query(conn, query)

,id,first_name,last_name,school,hire_date,salary
0,3,Samuel,Cole,Myers Middle School,2005-08-01,43500


3. Rank teachers hired since Jan 1, 2010, ordered by highest paid to lowest

In [5]:
query = """
SELECT *
FROM teachers
WHERE hire_date >= '2010-01-01'
ORDER BY salary DESC;
"""

execute_query(conn, query)

,id,first_name,last_name,school,hire_date,salary
0,6,Kathleen,Roush,F.D. Rossevelt HS,2010-10-22,38500
1,1,Janet,Smith,F.D. Rossevelt HS,2011-10-30,36200
2,4,Samantha,Bush,Myers Middle School,2011-10-30,36200


## Chapter 3: Understanding Data Types

Challenge Questions

1. Your company delivers fruit and vegetables to local grocery stores, and you need to track the mileaage driven by each driver each day to a tenth of a mile. Assuming no driver would ever travel more that 999 miles in a day, what would be an appropriate data type for the mileage column in your table. Why?

    Assuming we would only need to track to the nearest tenth of a mile and drivers never drive more than 999 miles, we would want a data type that reflects 4 significant figures. I would choose to use the NUMERIC data type with a precision of 4 and a scale of 1.

2. In the table listing each driver in your company, what are appropriate data types for the drivers' first and last names? Why is it a good idea to separate firs and last names into two columns rather than having one larger name column?

    I would choose the VARCHAR data type for the first and last name columns in my table with an ample amount of characters to suit the two fields - (15) and (30) respectively. This data type is best for minimizing space as it can go up to the indicated character count but if a first or last name does not have that many characters, the program will not pad/fill in the remaining values. It is not adviseable to combine both first and last names into one field for the purpose of saving space - the combined field may need to house extra characters like a space or comma to distinguish the two values - and because a combined field may make future queries harder to perform.

3. Assume you have a text column that includes strings formatted as dates. One of the strings is written as '4//2017'. What will happen when you try to convert that string to the timestamp data type?

    An ERROR message should pop up when trying to convert the text '4//2017' in to a TIMESTAMP as it is lacking the months and is not written in the format required by the TIMESTAMP data type - 'YYYY-MM-DD HH:MM:SS'.

## Chapter 4: Importing and Exporting Data

Challenge Questions

1. Write a WITH statement to include with COPY to handle the import of an imaginary text file whose first couple of rows look like this:

In [6]:
# ---------
# id:movie:actor
# 50:#Mission: Impossible#:Tom Cruise
# ---------

	COPY example_table
	FROM 'C:/RandomDirectory/indicatedtextfile.txt'
	WITH (FORMAT CSV, HEADER, DELIMITER ':', QUOTE '#');


2. Using the table us_counties_2010_ you created and filled in this chapter, export to a CSV file the 20 counties in the United States thathave the most housing units. Make sure you export only each county's name, state, and number of housing units.


	COPY (
		SELECT geo_name, state_us_abbreviation, housing_unit_count
		FROM us_counties_2010
		ORDER BY housing_unit_count DESC
		LIMIT 20
	)
	TO 'C:/RandomDirectory/housing_export.csv'
	WITH (FORMAT CSV, HEADER);

3. Imagine you're importing a file that contains a column with these values:

In [7]:
# ----
# 17519.668
# 20084.461
# 18973.335
# ----

3. Will a column in your target table with data type NUMERIC(3,8) work for these values?

    The data type NUMERIC(3,8) will not work as it has switched the precision and scale values. It should instead be NUMERIC(8,3) to indicate that there should be 8 total digits with only 3 of them being to the right of the decimal.

## Chapter 5: Basic Math And Stats With SQL

Challenge Questions

1. Write a SQL statement for calculating the area of a circle whose radius is 5 inches. Do you need parenthese in your calculation? Why or why not?


In [8]:
query = """
SELECT 3.14 * (5 ^ 2);
"""

execute_query(conn, query)

,?column?
0,78.5


    Answer: In this case we do not need parentheses as order of operations will give exponents priority over multiplication. However, parentheses may help make the expression easier to understand.

2. Using the  2010 Census county data, find out which New York state county has the highest percentage of the population that identified as "American Indian/Alaska Native Alone".

In [9]:
query = """
SELECT geo_name AS County,
	state_us_abbreviation,
	p0010001 AS total_population,
	p0010005 AS american_indian_alaska_native_alone,
	(CAST(p0010005 AS NUMERIC(8,1)) / p0010001) * 100 AS Pct_American_Indian
FROM us_counties_2010
WHERE state_us_abbreviation = 'NY'
ORDER BY Pct_American_Indian DESC
LIMIT 1;
"""

execute_query(conn, query)

,county,state_us_abbreviation,total_population,american_indian_alaska_native_alone,pct_american_indian
0,Franklin County,NY,51599,3797,7.35866974166166011000


3. Was the 2010 median county population higher in California or New York


In [10]:
query = """
SELECT state_us_abbreviation AS "State",
       percentile_cont(0.5)
	   WITHIN GROUP (ORDER BY p0010001) AS "Median"
FROM us_counties_2010
WHERE state_us_abbreviation IN ('NY', 'CA')
GROUP BY state_us_abbreviation;
"""

execute_query(conn, query)

,State,Median
0,CA,179140.5
1,NY,91301.0


     Answer: Based on the output from the query above, California had a higher median county population than New York - 179,140.5 to 91,301 respectively.

## Chapter 6: Joining Tables in a Relational Database

Challenge Questions

1. The table us_counties_2010 contains 3143 rows, and us_counties_2000 has 3141. That reflects the ongoing adjustments to county level geographies that typically result from government decision making. Using appropriate joins and the NULL value, identify which counties don't exist in both tables.


In [11]:
# Answer: The following query reveals that there are 5 counties from Alaska along with Broomfield county are not present in both tables.

query = """
SELECT
	ten.geo_name AS county,
	ten. state_us_abbreviation,
	twok.geo_name
FROM us_counties_2010 AS ten
LEFT JOIN us_counties_2000 AS twok
ON ten.state_fips = twok.state_fips
	AND ten.county_fips = twok.county_fips
WHERE twok.geo_name IS NULL;
"""

execute_query(conn, query)

,county,state_us_abbreviation,geo_name
0,Hoonah-Angoon Census Area,AK,None
1,Petersburg Census Area,AK,None
2,Prince of Wales-Hyder Census Area,AK,None
3,Skagway Municipality,AK,None
4,Wrangell City and Borough,AK,None
5,Broomfield County,CO,None


2. Using either the median() or precentile_cont() functions, determine the median of the percent change in county population.


In [12]:
query = """
SELECT
	PERCENTILE_CONT(.5) WITHIN GROUP (ORDER BY (ROUND((CAST(ten.p0010001 AS NUMERIC(8,1)) - twok.p0010001) / twok.p0010001 * 100, 1))) AS median_pop_change
FROM us_counties_2010 AS ten
JOIN us_counties_2000 AS twok
ON ten.state_fips = twok.state_fips
	AND ten.county_fips = twok.county_fips;
"""

execute_query(conn, query)

,median_pop_change
0,3.2


3. Which county had the greates percentage loss of population between 2000 and 2010?

In [13]:
# Answer: The following query shows that St. Bernard Parish had the greatese population loss.

query = """
SELECT c2010.geo_name AS County,
       c2010.state_us_abbreviation,
       c2010.p0010001 AS pop_2010,
       c2000.p0010001 AS pop_2000,
       c2010.p0010001 - c2000.p0010001 AS raw_change,
       round( (CAST(c2010.p0010001 AS DECIMAL(8,1)) - c2000.p0010001)
           / c2000.p0010001 * 100, 1 ) AS pct_change
FROM us_counties_2010 c2010 INNER JOIN us_counties_2000 c2000
ON c2010.state_fips = c2000.state_fips
   AND c2010.county_fips = c2000.county_fips
ORDER BY pct_change ASC;
"""

execute_query(conn, query)

,county,state_us_abbreviation,pop_2010,pop_2000,raw_change,pct_change
0,St. Bernard Parish,LA,35897,67229,-31332,-46.6
1,Kalawao County,HI,90,147,-57,-38.8
2,Issaquena County,MS,1406,2274,-868,-38.2
3,Cameron Parish,LA,6839,9991,-3152,-31.5
4,Orleans Parish,LA,343829,484674,-140845,-29.1
...,...,...,...,...,...,...
3132,Loudoun County,VA,312311,169599,142712,84.1
3133,Lincoln County,SD,44828,24131,20697,85.8
3134,Flagler County,FL,95696,49832,45864,92.0
3135,Pinal County,AZ,375770,179727,196043,109.1


## Chapter 7: Table Design That Works For You

    CREATE TABLE albums (
        album_id bigserial,
        album_catalog_code varchar(100),
        album_title text,
        album_artist text,
        album_time interval,
        album_release_date date,
        album_genre varchar(40),
        album_description text
    );

    CREATE TABLE songs (
        song_id bigserial,
        song_title text,
        song_artist text,
        album_id bigint
    );

Use the tables to answer these questions:

1. Modify these CREATE TABLE statements to include primary and foreign keys plus additional constraints on both tables.

    Answer: 
    
    CREATE TABLE albums (
        album_id bigserial,
        album_catalog_code varchar(80) NOT NULL,
        album_title text NOT NULL,
        album_artist text NOT NULL,
        album_release_date date,
        album_genre varchar(30),
        album_description text,
        CONSTRAINT album_id_key PRIMARY KEY (album_id),
        CONSTRAINT release_date_check CHECK (album_release_date > '1/1/1925')
    );

    CREATE TABLE songs (
        song_id bigserial,
        song_title text NOT NULL,
        song_artist text NOT NULL,
        album_id bigint REFERENCES albums (album_id),
        CONSTRAINT song_id_key PRIMARY KEY (song_id)
);

2. Instead of using column_id as a surrogate key for your primary key, are there any columns in albums that could be useful as a natural key? What would you have to know to decide?

    Answer: 
    
    Album_catalogue_code may be a viable primary key however we would have to know if it is unique across multiple companies or if it is always provided.


3. To speed up queries, which columns are good candidates for indexes?

    Answer: 
    
    Any column that is designated as a primary key should be indexed and so should columns used as foreign keys. In these tables, we should consider indexing the album_id, titles, artists, and album_release_date columns.

## Chapter 8: Extracting Information By Grouping and Summarizing

Challenge Questions

1. What is the pattern in the use of technology in libraries? Both the 2014 and 2009 survey tables contain the columns gpterms and pitusr. Write code to calculate the percent change in the sum of each column over time. Watch out for negative values

In [14]:
# Answer the following query should return the pct change in gpterms and pitusr
query = """
SELECT
    pls14.stabr,
    SUM(pls14.gpterms) AS gpterms_2014,
    SUM(pls09.gpterms) AS gpterms_2009,
    ROUND( (CAST(SUM(pls14.gpterms) AS DECIMAL(10, 1)) - SUM(pls09.gpterms)) /
        SUM(pls09.gpterms) * 100, 2) AS pct_change_gpterms,
    SUM(pls14.pitusr) AS pitusr_2014,
    SUM(pls09.pitusr) AS pitusr_2009,
    ROUND( (CAST(SUM(pls14.pitusr) AS DECIMAL(10, 1)) - SUM(pls09.pitusr)) /
        SUM(pls09.pitusr) * 100, 2) AS pct_change_pitusr
FROM pls_fy2014_pupld14a AS pls14
JOIN pls_fy2009_pupld09a AS pls09
    ON pls14.fscskey = pls09.fscskey
WHERE pls14.gpterms >=0 AND pls09.gpterms >= 0 AND pls14.pitusr >=0 AND pls09.pitusr >= 0
GROUP BY pls14.stabr
ORDER BY pct_change_gpterms DESC, pct_change_pitusr DESC
LIMIT 5
;
"""

execute_query(conn, query)

,stabr,gpterms_2014,gpterms_2009,pct_change_gpterms,pitusr_2014,pitusr_2009,pct_change_pitusr
0,GU,547,59,827.12,39842,19564,103.65
1,DC,1000,594,68.35,1050623,140251,649.10
2,AK,994,618,60.84,771075,1061498,-27.36
3,DE,772,487,58.52,622515,451689,37.82
4,ID,1792,1151,55.69,1878131,1986141,-5.44


2. Both library survey tables contain a column called obereg. Just as we calculated the percent change in visits grouped by state, do the same to group percent changes in visits by U.S. region using obereg. 

In [15]:
query = """
SELECT
    pls14.obereg,
    SUM(pls14.visits) AS visits_2014,
    SUM(pls09.visits) AS visits_2009,
    ROUND( (CAST(SUM(pls14.visits) AS DECIMAL(10, 1)) - SUM(pls09.visits)) /
        SUM(pls09.visits) * 100, 2) AS pct_change
FROM pls_fy2014_pupld14a AS pls14
JOIN pls_fy2009_pupld09a AS pls09
    ON pls14.fscskey = pls09.fscskey
WHERE pls14.visits >=0 AND pls09.visits >= 0
GROUP BY pls14.obereg
HAVING SUM(pls14.visits) > 50000000
ORDER BY pct_change DESC
LIMIT 5
;
"""

execute_query(conn, query)

,obereg,visits_2014,visits_2009,pct_change
0,07,67482370,68703182,-1.78
1,01,87441083,92090682,-5.05
2,04,100458571,110484975,-9.07
3,08,244298987,271264003,-9.94
4,02,232953540,261305024,-10.85


3. Which join type will show you all the rows in both tables, including those without a match? Write such a query and add an IS NULL filter in a WHERE clause to show agencies not included in one or the other table.

    A full outer join will show rows from both tables in a join, regardless of if they are matches. Below is a query that will show the agencies not included in one or the other table:

In [16]:
query = """
SELECT
    pls14.stabr AS state_14,
    pls09.stabr AS state_09
FROM pls_fy2014_pupld14a AS pls14
FULL OUTER JOIN pls_fy2009_pupld09a AS pls09
    ON pls14.fscskey = pls09.fscskey
WHERE pls14.stabr IS NULL OR pls09.stabr IS NULL
ORDER BY pls14.stabr DESC
LIMIT 15
;
"""

execute_query(conn, query)

,state_14,state_09
0,None,TX
1,None,NJ
2,None,PR
3,None,TX
4,None,VT
5,None,NC
6,None,NE
7,None,PR
8,None,PR
9,None,NH


## Chapter 9: Inspecting and Modifying Data

Challenge Questions

The following questions will use the meat_poultry_egg_inspect table to try and answer the following questions: how many of the plants in the table processs meat, and how many process poultry?

1. Create two new columns called meat_processing and poultry_processing in your meat_poultry_egg_inspect table. Each can be of the type boolean.

    The following queries should create two columns in our meat_poultry_egg_inspect table:

        ALTER TABLE meat_poultry_egg_inspect ADD COLUMN meat_processing BOOLEAN;
        ALTER TABLE meat_poultry_egg_inspect ADD COLUMN poultry_processing BOOLEAN;
2. Using UPDATE, set meat_processing = TRUE on any row where the activities column contains the text "Meat Processing". Do the same update on the poultry processing column but this time look for the the text "Poultry Processing" in activities.

    The following queries should make the updates to our table described above:

In [17]:
query = """
UPDATE meat_poultry_egg_inspect
SET meat_processing = TRUE
WHERE activities ILIKE '%Meat Processing%';

UPDATE meat_poultry_egg_inspect
SET poultry_processing = TRUE
WHERE activities ILIKE '%Poultry Processing%';

SELECT *
FROM meat_poultry_egg_inspect
LIMIT 20;
"""

execute_query(conn, query)

,est_number,company,street,city,st,zip,phone,grant_date,activities,dbas,st_copy,company_standard,inspection_date,reviewed_date,meat_processing,poultry_processing
0,M1544,J & M Meat Co.,545 9th St.,Oakland,CA,94607,(510) 839-0380,1982-05-24,Meat Processing,None,CA,J & M Meat Co.,None,2018-02-07,True,None
1,V3507,Zollinger Cold Storage Corporation,699 West 1700 South,Logan,UT,84321,(801) 753-1134,1990-02-27,"Certification - Export, Identification - Meat,...",None,UT,Zollinger Cold Storage Corporation,None,2018-02-07,None,None
2,M34258,Beatrice Meat Company LLC,2595 Grant Blvd.,Beatrice,AL,36425,(251) 575-4021,2007-08-23,Meat Processing,None,AL,Beatrice Meat Company LLC,None,2018-02-07,True,None
3,V971,Supervalu Inc.,1400 West Gadsden Street,Quincy,FL,32351,(850) 875-2600,2011-09-16,"Certification - Export, Identification - Meat,...",None,FL,Supervalu Inc.,None,2018-02-07,None,None
4,M34092+V34092,"Plumrose USA, INC",125 Coley Road,Tupelo,MS,38801,(662) 840-6831,2011-11-02,"Certification - Export, Certification - Trichi...",None,MS,"Plumrose USA, INC",None,2018-02-07,True,None
5,M34617,Ellingson Companies Inc.,210 Main Street North,Deer Park,WI,54007,(715) 269-5118,2016-11-22,Meat Processing,Deers Food Locker,WI,Ellingson Companies Inc.,None,2018-02-07,True,None
6,M27472,"Noah's Ark Processors, LLC",1009 West M. Street,Hastings,NE,68901,(402) 461-3412,2015-03-05,"Meat Processing, Meat Slaughter",WR RESERVE,NE,"Noah's Ark Processors, LLC",None,2018-02-07,True,None
7,M20856+V20856,"Eureka Locker, Inc.",110 4-H Park Rd.,Eureka,IL,61530,(309) 467-2731,2011-07-01,"Meat Processing, Meat Slaughter, Voluntary Pro...",None,IL,"Eureka Locker, Inc.",None,2018-02-07,True,None
8,M4010A,"Euro Foods Inc., d/b/a Citterio USA Corp",718 Hazel Street,Freeland,PA,18224,(570) 636-3171,2016-07-19,Meat Processing,Citterio USA Corp.,PA,"Euro Foods Inc., d/b/a Citterio USA Corp",None,2018-02-07,True,None
9,M1673,Evans Food Products,4118 South Halsted Street,Chicago,IL,60609,(773) 254-7400,2016-08-19,Meat Processing,"Evans Foods Products , Inc.",IL,Evans Food Products,None,2018-02-07,True,None


3. Use the data from the new, update columns to count how many plants perform each type of activity. For a bonus activity, count how many plants perform both activities.

    The following queries should give us the count of plants that process meat, poultry, or both:

In [18]:
query = """
SELECT 
    COUNT(meat_processing) AS count_of_meat_plants,
    COUNT(poultry_processing) AS count_of_poultry_plants
FROM meat_poultry_egg_inspect
"""

execute_query(conn, query)

,count_of_meat_plants,count_of_poultry_plants
0,4764,3728


In [19]:
query = """
SELECT COUNT(*) as plants_who_process_both
FROM meat_poultry_egg_inspect
WHERE meat_processing = TRUE AND
      poultry_processing = TRUE;
"""

execute_query(conn, query)

,plants_who_process_both
0,3338


## Chapter 10: Statistical Funtions in SQL

1. IN this chapter we saw that the correlation coefficient, or r value, of the variables pct_bachelors_higher and median_hh_income was about 0.68. Write a query using the acs data set to show the correlation between pct_masters_higher and median_hh_income. Is the r value higher or lower? What might explain the difference?

    Answer: The result from our query below shows that the correlation coefficient between the variables pct_masters_higher and median_hh_income is roughly 0.57. This r value is lower than the bachelors_income_r calculated earlier in the chapter. This may be a weaker correlation coefficient because it takes more money/time to earn a masters degree or higher and therefore most households will not have an adult who possesses that level of education.

In [20]:
# The following query should show us the correlation between pct_masters_higher and median_hh_income
query = """
SELECT CORR(median_hh_income, pct_masters_higher) AS masters_income_r
FROM acs_2011_2015_stats;
"""

execute_query(conn, query)

,masters_income_r
0,0.5682


2. In the FBI crime data, which cities with a population of 500k or more have the highest rates of motor vehicle thefts (column motor_vehicle_theft)? Which have the highest violent crime rates (column violent_crime)?

    Answer: As shown by our queries below, Milwaukee has the highest rate of motor vehicle theft(12.3%) and Detroit has the highest rate of violent crimes (17.6%).

In [21]:
# Finding highest rate of motor vehicle thefts
query = """
SELECT
    city,
    st,
    population,
    motor_vehicle_theft,
    ROUND((motor_vehicle_theft::NUMERIC / population) * 1000, 1) AS pc_per_1000
FROM fbi_crime_data_2015
WHERE population >= 500000
ORDER BY pc_per_1000 DESC
LIMIT 10;
"""

execute_query(conn, query)


,city,st,population,motor_vehicle_theft,pc_per_1000
0,Milwaukee,Wisconsin,600400,7380,12.3
1,Albuquerque,New Mexico,559721,5179,9.3
2,Baltimore,Maryland,621252,5526,8.9
3,San Francisco,California,863782,6915,8.0
4,Detroit,Michigan,673225,5216,7.7
5,San Jose,California,1031458,7001,6.8
6,Denver,Colorado,682418,4365,6.4
7,Houston,Texas,2275221,13560,6.0
8,Indianapolis,Indiana,863675,4991,5.8
9,Dallas,Texas,1301977,7594,5.8


In [22]:
# Finding highest rate of violent crime rates
query = """
SELECT
    city,
    st,
    population,
    violent_crime,
    ROUND((violent_crime::NUMERIC / population) * 1000, 1) AS pc_per_1000
FROM fbi_crime_data_2015
WHERE population >= 500000
ORDER BY pc_per_1000 DESC
LIMIT 10;
"""

execute_query(conn, query)

,city,st,population,violent_crime,pc_per_1000
0,Detroit,Michigan,673225,11846,17.6
1,Memphis,Tennessee,657936,11449,17.4
2,Milwaukee,Wisconsin,600400,9583,16.0
3,Baltimore,Maryland,621252,9542,15.4
4,Indianapolis,Indiana,863675,11124,12.9
5,Washington,District of Columbia,672228,8084,12.0
6,Nashville Metropolitan,Tennessee,658029,7245,11.0
7,Philadelphia,Pennsylvania,1567810,16132,10.3
8,Houston,Texas,2275221,21994,9.7
9,Albuquerque,New Mexico,559721,5406,9.7


3. Revisit the libraries data in the table pls_fy_2014. Rank library agencies based on the rate of visits per 1000 population (column popu_lsa), and limit the query to agencies serving 250,000 people or more

In [23]:
query = """
SELECT
    libname,
    popu_lsa,
    ROUND(
        (visits::NUMERIC / popu_lsa) * 1000, 1
    ) AS visits_per_1000,
    RANK() OVER(ORDER BY  (visits::NUMERIC / popu_lsa) * 1000 DESC) AS visits_rank
FROM pls_fy2014_pupld14a
WHERE visits >= 0 AND
    popu_lsa >= 250000;
"""

execute_query(conn, query)

,libname,popu_lsa,visits_per_1000,visits_rank
0,CUYAHOGA COUNTY PUBLIC LIBRARY,616527,12962.8,1
1,SEATTLE PUBLIC LIBRARY,640500,10273.9,2
2,CENTRAL RAPPAHANNOCK REGIONAL LIBRARY,298007,10147.0,3
3,HOWARD COUNTY LIBRARY SYSTEM,293142,9893.5,4
4,ALAMEDA COUNTY LIBRARY,557378,8711.8,5
...,...,...,...,...
192,SANTA ANA PUBLIC LIBRARY,331953,773.4,193
193,MONT CO-NORRISTOWN PUB LIB,325368,529.1,194
194,MORRIS COUNTY LIBRARY,492276,442.0,195
195,PINAL COUNTY LIBRARY DISTRICT,396237,355.8,196


## Chapter 11: Working With Dates and Times

Challenge Questions

    1. Using the NY City taxi data, calculate the length of each ride using the pickup and drop-off timestamps. Sort the query results from the longest ride to the shortest. Do you notice anything about the longest or shortest trips that you might want to ask city officials about?

In [24]:
query = """
SELECT
    trip_distance,
    total_amount,
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    tpep_dropoff_datetime - tpep_pickup_datetime AS length
FROM nyc_yellow_taxi_trips_2016_06_01
ORDER BY length DESC
LIMIT 5
"""

execute_query(conn,query)

,trip_distance,total_amount,tpep_pickup_datetime,tpep_dropoff_datetime,length
0,2.40,14.30,2016-06-01 08:38:03-04:00,2016-06-02 08:37:40-04:00,0 days 23:59:37
1,1.38,11.84,2016-06-01 21:02:23-04:00,2016-06-02 21:01:54-04:00,0 days 23:59:31
2,0.00,0.00,2016-06-01 16:22:04-04:00,2016-06-02 16:21:29-04:00,0 days 23:59:25
3,2.39,17.38,2016-06-01 11:03:44-04:00,2016-06-02 11:03:06-04:00,0 days 23:59:22
4,1.44,8.80,2016-06-01 15:25:57-04:00,2016-06-02 15:25:17-04:00,0 days 23:59:20


In [25]:
query = """
SELECT
    trip_distance,
    total_amount,
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    tpep_dropoff_datetime - tpep_pickup_datetime AS length
FROM nyc_yellow_taxi_trips_2016_06_01
ORDER BY length
LIMIT 5
"""

execute_query(conn,query)

,trip_distance,total_amount,tpep_pickup_datetime,tpep_dropoff_datetime,length
0,0.00,70.00,2016-06-01 21:38:54-04:00,2016-06-01 21:34:21-04:00,-1 days +23:55:27
1,2.30,13.80,2016-06-01 15:46:15-04:00,2016-06-01 15:45:55-04:00,-1 days +23:59:40
2,0.00,3.30,2016-06-01 09:05:34-04:00,2016-06-01 09:05:34-04:00,0 days 00:00:00
3,0.00,3.30,2016-06-01 07:53:38-04:00,2016-06-01 07:53:38-04:00,0 days 00:00:00
4,0.00,0.00,2016-06-01 08:04:42-04:00,2016-06-01 08:04:42-04:00,0 days 00:00:00


    It looks as though the data for the shortest rides based on timestamps may have been entered incorrectly as the numbers show that there was a negative ride time which is not possible. Moreover there are taxi rides that are almost a day long which also seems very unusual.

2. Using the AT TIME ZONE keyworkds, write a query that displays the date and time for London, Johannesburg, Moscow, and Melbourne the moment January 1, 2100, arrives in New York City.

In [26]:
query = """
SELECT
    MAKE_TIMESTAMPTZ(2100, 1, 1, 0, 0, 0, 'US/Eastern') AT TIME ZONE 'US/Eastern' AS NYC,
    MAKE_TIMESTAMPTZ(2100, 1, 1, 0, 0, 0, 'US/Eastern') AT TIME ZONE 'Europe/London' AS London,
    MAKE_TIMESTAMPTZ(2100, 1, 1, 0, 0, 0, 'US/Eastern') AT TIME ZONE 'Africa/Johannesburg' AS Johannesburg,
    MAKE_TIMESTAMPTZ(2100, 1, 1, 0, 0, 0, 'US/Eastern') AT TIME ZONE 'Europe/Moscow' AS Moscow,
    MAKE_TIMESTAMPTZ(2100, 1, 1, 0, 0, 0, 'US/Eastern') AT TIME ZONE 'Australia/Melbourne' AS Melbourne;
"""

execute_query(conn, query)

,nyc,london,johannesburg,moscow,melbourne
0,2100-01-01,2100-01-01 05:00:00,2100-01-01 07:00:00,2100-01-01 08:00:00,2100-01-01 16:00:00


3. Use the statistics funcitons from Chapter 10 to calculate the correlation coefficient and r-squared values using trip time and the total_amount column in the New York City taxi data, which represents the total amount charged to passengers. Do the samee with the trip_distance and total_amount columns. Limit the query to rides that last 3 hours or less.

In [27]:
# Query to find the correlation between length of trip and total cost of a taxi ride
query = """
SELECT
    ROUND(
          CORR(total_amount, (
              DATE_PART('epoch', tpep_dropoff_datetime) -
              DATE_PART('epoch', tpep_pickup_datetime)
                ))::NUMERIC, 2
          ) AS amount_time_corr,
    ROUND(
        REGR_R2(total_amount, (
              DATE_PART('epoch', tpep_dropoff_datetime) -
              DATE_PART('epoch', tpep_pickup_datetime)
        ))::NUMERIC, 2
    ) AS amount_time_r2
FROM nyc_yellow_taxi_trips_2016_06_01
WHERE tpep_dropoff_datetime - tpep_pickup_datetime <= '03:00:00';
"""

execute_query(conn, query)

,amount_time_corr,amount_time_r2
0,0.80,0.64


In [28]:
# Query to find the correlation between distance and total cost of a taxi ride
query = """
SELECT
    CORR(trip_distance, total_amount) AS cost_amount_r,
    REGR_R2(trip_distance, total_amount) AS cost_amount_r2
FROM nyc_yellow_taxi_trips_2016_06_01
WHERE tpep_dropoff_datetime - tpep_pickup_datetime <= '03:00:00';
"""

execute_query(conn, query)

,cost_amount_r,cost_amount_r2
0,0.855132,0.731251


## Chapter 12: Advanced Query Techniques

Challenge Questions:

1. Revise the temperature_readings work from the chapter to dig deeper into the nuances of Waikiki's high temperatures. Limit the temps_collapsed table to the Waikiki maximum daily temperature observations. Then use the WHEN clause in the CASE statement to reclassify the temperatures into seven gorups that would result in the following text output

    '90 or more'
    '88-89'
    '86-87'
    '84-85'
    '82-83'
    '80-81'
    '79 or less'
In which of those groups does Waikiki's daily maximum temperature fall most often?

In [29]:
# The following query shows that Waikiki's daily maximum temperature tends to fall in the 86-87 degree category most often
query = """
WITH temps_collapsed AS
    (SELECT
        station_name,
        CASE WHEN max_temp >= 90 THEN '90 or more'
        WHEN max_temp BETWEEN 88 AND 89 THEN '88-89'
        WHEN max_temp BETWEEN 86 AND 87 THEN '86-87'
        WHEN max_temp BETWEEN 84 AND 85 THEN '84-85'
        WHEN max_temp BETWEEN 82 AND 83 THEN '82-83'
        WHEN max_temp BETWEEN 80 AND 81 THEN '80-81'
        ELSE '79 or less'
        END AS max_temperature_group
    FROM temperature_readings
    WHERE station_name = 'WAIKIKI 717.2 HI US')

SELECT station_name, max_temperature_group, COUNT(*)
FROM temps_collapsed
GROUP BY station_name, max_temperature_group
ORDER BY station_name, COUNT(*) DESC;
"""

execute_query(conn, query)

,station_name,max_temperature_group,count
0,WAIKIKI 717.2 HI US,86-87,118
1,WAIKIKI 717.2 HI US,84-85,89
2,WAIKIKI 717.2 HI US,88-89,62
3,WAIKIKI 717.2 HI US,82-83,61
4,WAIKIKI 717.2 HI US,80-81,23
5,WAIKIKI 717.2 HI US,79 or less,8
6,WAIKIKI 717.2 HI US,90 or more,5


2. Revise the ice cream survey crosstab from the chapter to flip the table. In other words, make flavor the rows and office columns. Which elements of the query do you need to change? Are the counts different?

In [30]:
# The following query shows the transformed Crosstab for the ice cream survey data
query = """
SELECT *
FROM CROSSTAB(
        'SELECT flavor,
            office,
            COUNT(*)
        FROM ice_cream_survey
        GROUP BY flavor, office
        ORDER BY flavor',

        'SELECT office
        FROM ice_cream_survey
        GROUP BY office
        ORDER BY office'
        )
    AS (flavor VARCHAR(20),
        downtown BIGINT,
        midtown BIGINT,
        uptown BIGINT
        );
"""

execute_query(conn, query)

,flavor,downtown,midtown,uptown
0,Chocolate,23,41.0,22
1,Strawberry,32,NaN,17
2,Vanilla,19,23.0,23
